In [7]:
import pandas as pd

pc=pd.read_csv('postcodes_sampled.csv')
pc.head()

,postcode,sector,easting,northing,localAuthority,altitude,soilType
0,PO7 8PR,PO7 8,469395.0,108803.0,Havant,30,Planosols
1,SO17 1NS,SO17 1,442771.0,114321.0,Southampton,20,Unsurveyed/Urban
2,TN28 8XN,TN28 8,606861.0,124689.0,Folkestone and Hythe,10,Cambisols
3,KT3 4JW,KT3 4,521649.0,168848.0,Kingston upon Thames,20,Unsurveyed/Urban
4,CT2 8AA,CT2 8,614532.0,158074.0,Canterbury,10,Unsurveyed/Urban


In [8]:
Look_up_table = pc[['postcode', 'localAuthority']]

Look_up_table.set_index(['postcode'], inplace=True)

Look_up_table


,localAuthority
postcode,
PO7 8PR,Havant
SO17 1NS,Southampton
TN28 8XN,Folkestone and Hythe
KT3 4JW,Kingston upon Thames
CT2 8AA,Canterbury
...,...
SE22 8BE,Southwark
SW10 0JB,Kensington and Chelsea
HP21 9QS,Buckinghamshire


In [9]:
LocalAuthority_read = Look_up_table[Look_up_table.index.str.startswith('PO7 8PR')]['localAuthority'][0]

LocalAuthority_read

'Havant'

In [10]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Encode=LabelEncoder()
local_Authority_encoded=Encode.fit_transform(pc['localAuthority'])

local_Authority_encoded

array([41, 72, 30, ..., 13, 83, 14])

In [11]:
LE = LabelEncoder()
postcode_encoded = LE.fit_transform(pc[['postcode']])

postcode_encoded 


/opt/homebrew/Caskroom/miniconda/base/envs/dsml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([21070, 29605, 35364, ..., 11849, 34587, 39863])

In [12]:
from sklearn.model_selection import train_test_split

Y = pd.DataFrame(local_Authority_encoded)
X = pd.DataFrame(postcode_encoded)

X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size=0.3, random_state=1)


In [13]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, Y_train)
Y_pred = model.predict(X_test)

model.score(X_test, Y_test)



0.06197716826335398

In [19]:
def get_easting_northing(self, postcodes):
        """Get a frame of OS eastings and northings from a collection
        of input postcodes.

        Parameters
        ----------

        postcodes: sequence of strs
            Sequence of postcodes.

        Returns
        -------

        pandas.DataFrame
            DataFrame containing only OSGB36 easthing and northing indexed
            by the input postcodes. Invalid postcodes (i.e. not in the
            input unlabelled postcodes file) return as NaN.
         """

        frame = self.postcodedb.copy()
        frame = frame.set_index('postcode')

        return frame.loc[postcodes, ['easting', 'northing']]

In [14]:
class LocalAuthorityModel:

    def __init__(self, path, method):
        self.X_train, self.X_test, self.y_train, self.y_test = self.load_data(path)
        self.model = self.create_pipeline(method)
        self.model = self.train_model()
        return None

    def load_data(self, path):
        # Load data
        data = pd.read_csv(path)
        data = data.drop_duplicates()
        y = data.localAuthority
        print(y.shape)


        X = data[['postcodes']]
        print(X.shape)

        y = LabelEncoder().fit_transform(y)

        return train_test_split(X, y, train_size=0.8)

In [17]:
def create_pipeline(self, method):
        method_dict = {1: KNeighborsClassifier()}
        pipe = Pipeline([
            ('scaler', None),
            ('model', method_dict[method])
        ])

        return pipe

def train_model(self):
        grid_dict = {'scaler':[None, MinMaxScaler()],
                    'model__n_neighbors': [1,2,3,5,10]}

        search = GridSearchCV(self.model, grid_dict, cv=5)
        search.fit(self.X_train, self.y_train)

        return search.best_estimator_

def predict(self, postcodes):
        new_samples = pd.DataFrame([postcodes], index=['postcodes']).T
        print(new_samples)
        pred = self.model.predict(new_samples)

        return pd.Series(pred, index=[(est, nth) for est, nth in
                                    zip(postcodes)],
                             name='localAuthority')
